# Loading and Updating Data in the Cytoxnet Package

While the cytoxnet package come pre-loaded with a handfull of datasets, you'll probably want to train the models using your own datasets.

The cytoxnet package utilizes sqlite to create a lightweight, portable database for data storage and retreival.  It contains a codex of molecular compounds which is updated with any new smiles strings anytime a new dataset is loaded into the database.

In this notebook, we're look at adding new datasets to the database as well as updating existing datasets when needed.

## Loading Data
Let's start by importing the packages we need to add new data

In [1]:
import cytoxnet.dataprep.featurize as ft
import cytoxnet.dataprep.dataprep as dp
import cytoxnet.data as data
import cytoxnet.dataprep.database as db
import cytoxnet.dataprep.io

In [24]:
import sqlite3
import sqlalchemy
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Float, Boolean, DateTime, Interval, MetaData, create_engine, select, or_, and_, join, union
import pandas as pd
import csv
import os


import os
import pkg_resources

import numpy as np
import pandas as pd
import deepchem as dc



Next, let's create the compound codex.  This is a persistent table which will be updated with new smile strings every time we add a new dataset to the database folder.  It is also where we will store feature objects which we can retreive later.  Saving the feature objects this way saves us computational time since we won't have to generate feature objects every time we want to retreive a dataset.

In [3]:
cytoxnet.dataprep.io.create_compound_codex()

Running the create_compound_codex function before we've added any new datasets will result in an empty csv file, which we cand see by calling the newly generated csv file as a Pandas dataframe.

In [4]:
compounds_df = pd.read_csv('database/compounds.csv')
compounds_df

,Unnamed: 0,smiles


Now, let's add some new datasets and see what happens.  We'll start by loading in a couple of the datasets which come with the package (the lunghini algea EC50 and daphnia EC50 datasets).  We can load these datasets using the add_datasets function from the io.py file.

After running the add_datasets function, you might notice it takes a while to complete.  Don't worry!  Generating features can take a while.  In this example, we'll run the Circular Fingerprint featurizer since it's one of the more lightweight featurizers.

In [5]:
dataframe_dict = cytoxnet.dataprep.io.add_datasets(['lunghini_algea_EC50', 'lunghini_daphnia_EC50'], names=['algea', 'daphnia'], new_featurizers=['CircularFingerprint'])

The add_datasets function creates a dictionary with table names as the keys and dataframe objects as the values.  Let's take a look at some of the data from the algea dataset which we've named 'algea'.  When you load in your own datasets, you can of course call the datasets anything you want!

In [6]:
dataframe_dict['algea']

,chemical_formula,smiles,casnum,molecular_weight,species,algea_EC50,units,source,foreign_key
0,C10H10Br2O2,BrC(Br)c1ccccc1OCC1CO1,30171-80-3,321.993195,algea,-0.879477,log(mg/L),"NITE, Literature set",0
1,C8H7Br,BrC=Cc1ccccc1,103-64-0,183.045181,algea,3.919991,log(mg/L),ECHA,1
2,C9H15Br6O4P,O=P(OCC(Br)CBr)(OCC(Br)CBr)OCC(Br)CBr,126-72-7,697.610779,algea,0.875469,log(mg/L),"NITE, ECOTOX, OASIS, Literature set",2
3,C9H9Br,BrCC=Cc1ccccc1,4392-24-9,197.071762,algea,2.940220,log(mg/L),Literature set,3
4,C2H4Br2,BrCCBr,106-93-4,187.861160,algea,3.255786,log(mg/L),"ECHA, ECOTOX",4
...,...,...,...,...,...,...,...,...,...
1435,C4H4S,c1ccsc1,110-02-1,84.139557,algea,4.382027,log(mg/L),"NITE, VEGA, Literature set",1435
1436,C7H6N2,c1ccc2[nH]cnc2c1,51-17-2,118.135941,algea,3.288402,log(mg/L),ECHA,1436
1437,C7H5NS,c1ccc2scnc2c1,95-16-9,135.186295,algea,3.885679,log(mg/L),"ECHA, NITE, ECOTOX, OASIS, Literature set, VEGA",1437
1438,C2H3N3,c1nc[nH]n1,288-88-0,69.065323,algea,3.942552,log(mg/L),"ECHA, ECOTOX",1438


Beyond creating a dataframe dictionary, the add_datasets function updates the compounds codex which we created early all of the new smiles appearing in our algea and daphnia datasets.  It also assigns a unique integer id number to each smiles and generates any feature objects which we specify.  In this example, we have chosen to generate just the circular fingerprints since they are generally small and can be generated relatively quickly, but we could also generate more than one feature at once if we chose.

If we look back at the previously empty compounds codex, we can see now that it has been populated with all our new smiles and circular fingerprint values!

In [7]:
compounds_df = pd.read_csv('database/compounds.csv')
compounds_df

,Unnamed: 0,smiles,CircularFingerprint
0,0,BrC(Br)c1ccccc1OCC1CO1,[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1,1,BrC=Cc1ccccc1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2,2,O=P(OCC(Br)CBr)(OCC(Br)CBr)OCC(Br)CBr,[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3,3,BrCC=Cc1ccccc1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4,4,BrCCBr,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
...,...,...,...
2678,2678,c1ccc2cnccc2c1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2679,2679,c1ccc2nc3ccccc3cc2c1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2680,2680,c1ccncc1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2681,2681,c1coc(-c2nc3ccccc3[nH]2)c1,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


Before moving on, we should send our updated compounds table into the our persistent database.  Since we're loading in our codex, we'll set the codex parameter to True.

In [32]:
cytoxnet.dataprep.db.table_creator(table_name='compounds', dataframe=compounds_df, codex=True, id_col=None)

2021-06-05 14:06:35,549 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-05 14:06:35,552 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("compounds")
2021-06-05 14:06:35,555 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:06:35,562 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("compounds")
2021-06-05 14:06:35,565 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:06:35,570 INFO sqlalchemy.engine.Engine 
CREATE TABLE compounds (
	ids INTEGER NOT NULL, 
	smiles VARCHAR, 
	circularfingerprint VARCHAR, 
	PRIMARY KEY (ids)
)


2021-06-05 14:06:35,575 INFO sqlalchemy.engine.Engine [no key 0.00473s] ()
2021-06-05 14:06:35,597 INFO sqlalchemy.engine.Engine COMMIT
2021-06-05 14:06:35,666 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("compounds")
2021-06-05 14:06:35,668 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:06:35,688 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-05 14:06:35,749 INFO sqlalchemy.engine.Engine INSERT INTO compounds

In order to create persistant instances of our new algea and daphnia datasets, we can load them into the cytoxnet.db database file.  This database is built using sqlite so that you can store the entire database locally.

We'll add our datasets to the database using the table_creator and tables_to_database functions.  The table_creator function takes in a table name and a dataframe object.  We can add load in each table individually...

In [33]:
cytoxnet.dataprep.db.table_creator(table_name='algea', dataframe=dataframe_dict['algea'], codex=False, id_col=None)

ValueError: cannot insert ids, already exists

... or we can iterate through the dataframe dictionary we created a few cells back using the tables_to_database function.  This function simply calls the table_creator function and iterates through a dictionary object containing table names and dataframe objects.  This is expecially advantagous if we have multiple new datasets we're interested in, since we can quickly load them all into the database using the add_dataset function together with the tables_to_databae function.

In [37]:
cytoxnet.dataprep.db.tables_to_database(dataframe_dict=dataframe_dict)

2021-06-05 14:07:37,098 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("compounds")
2021-06-05 14:07:37,099 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:07:37,102 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-05 14:07:37,103 INFO sqlalchemy.engine.Engine [raw sql] ('compounds',)
2021-06-05 14:07:37,106 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("compounds")
2021-06-05 14:07:37,107 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:07:37,109 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("compounds")
2021-06-05 14:07:37,110 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-05 14:07:37,113 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-06-05 14:07:37,121 INFO sqlalchemy.engine.Engine [raw sql] ('compounds',

2021-06-05 14:07:37,655 INFO sqlalchemy.engine.Engine [generated in 0.08760s] ((0, 'C=C(Br)c1ccccc1', 'C8H7Br', '98-81-7', 183.0451812744141, 'daphnia', -1.1086626245216111, 'log(mg/L)', 'NITE, VEGA, Literature set', 1440), (1, 'BrC(Br)Br', 'CHBr3', '75252', 252.7306365966797, 'daphnia', 3.828361036331304, 'log(mg/L)', 'ECOTOX, OASIS, VEGA, Literature set', 1441), (2, 'BrC(Br)c1ccccc1OCC1CO1', 'C10H10Br2O2', '30171-80-3', 321.9931945800781, 'daphnia', 0.262364264467491, 'log(mg/L)', 'NITE, Literature set', 0), (3, 'BrC1CCCCC1', 'C6H11Br', '108850', 163.0555419921875, 'daphnia', 3.044522437723423, 'log(mg/L)', 'ECOTOX, OASIS, VEGA, Literature set', 1442), (4, 'BrC=C(Br)Br', 'C2HBr3', '598163', 264.7413330078125, 'daphnia', 2.516264483437345, 'log(mg/L)', 'ECOTOX, OASIS, VEGA, Literature set', 1443), (5, 'BrC=Cc1ccccc1', 'C8H7Br', '103-64-0', 183.0451812744141, 'daphnia', 1.8082887711792648, 'log(mg/L)', 'ECHA', 1), (6, 'N#CCCC(Br)(C#N)CBr', 'C6H6Br2N2', '35691-65-7', 265.9332275390625, 

Any time you make updates to your csv files or want to add a new one to train your model with, all you need to do is execute these few steps, and the cytoxnet database will automatically update the compounds codex file and add all the data to the persistant database!